# Tugas 2

Prio Budi Laksono

210411100177

Preprocessing hasil crawling data dari jatim.tribunnews.com

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

# Fungsi untuk mendapatkan konten dari URL
def get_soup(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')
    except requests.RequestException as e:
        print(f"Kesalahan saat mengambil {url}: {e}")
        return None

# Fungsi untuk mendapatkan detail artikel dari halaman detail
def get_article_details(detail_url):
    detail_soup = get_soup(detail_url)
    if detail_soup:
        # Ambil isi berita
        content = ' '.join([p.text for p in detail_soup.find_all('p')])

        # Ambil tanggal publikasi
        date_tag = detail_soup.find('time')
        date = date_tag.text.strip() if date_tag else 'Tidak ada tanggal'

        # Ambil judul berita
        title_tag = detail_soup.find('h1')
        title = title_tag.text.strip() if title_tag else 'Tidak ada judul'

        # Ambil kategori dari breadcrumb
        breadcrumb = detail_soup.find('ul', {'class': 'breadcrumb'})
        category = breadcrumb.find_all('li')[-1].find('span').text.strip() if breadcrumb else 'Tidak ada kategori'

        return {
            'judul': title,
            'isi_berita': content,
            'tanggal': date,
            'kategori': category,
            'url': detail_url
        }
    return None

# Fungsi untuk mendapatkan artikel dari suatu kategori
def get_articles(category_url, category_name, max_articles=100):
    articles = []
    page = 1

    while len(articles) < max_articles:
        url = f'{category_url}?page={page}'  # Periksa pola yang benar untuk paginasi
        print(f"Mengambil: {url}")  # Output debug
        soup = get_soup(url)

        if soup is None:
            break

        # Cari artikel di halaman
        article_list = soup.find_all('h3')  # Berdasarkan tata letak yang diamati

        if not article_list:
            print(f"Tidak ada artikel ditemukan di halaman {page}.")
            break

        for article in article_list:
            if len(articles) >= max_articles:
                break

            # Ambil URL detail artikel
            title_tag = article.find('a')
            detail_url = title_tag['href'] if title_tag else None

            if detail_url:
                if not detail_url.startswith('http'):
                    detail_url = f'https://jatim.tribunnews.com{detail_url}'
                # Buka halaman detail artikel
                article_details = get_article_details(detail_url)
                if article_details:
                    # Tambahkan ke daftar artikel
                    articles.append(article_details)

        page += 1
        time.sleep(2)  # Beri jeda agar tidak terlalu cepat melakukan permintaan

    return articles

# URL Kategori
categories = {
    'Travel': 'https://jatim.tribunnews.com/travel',
    'Sport': 'https://jatim.tribunnews.com/sport'
}

# Mengumpulkan semua data
all_articles = []
for category_name, category_url in categories.items():
    print(f"Menambang kategori {category_name}...")
    articles = get_articles(category_url, category_name, max_articles=100)
    all_articles.extend(articles)

# Simpan ke dalam DataFrame
df = pd.DataFrame(all_articles)

# Debug: Memeriksa apakah data berhasil diambil
print(f"Jumlah artikel yang diambil: {len(df)}")
if len(df) == 0:
    print("Peringatan: Tidak ada data yang dikumpulkan, pastikan proses scraping berhasil.")

# Simpan ke dalam file CSV hanya jika ada data
if not df.empty:
    file_path = 'tribunnews_articles.csv'
    df.to_csv(file_path, index=False)

    # Debug: Memastikan file CSV tersimpan dengan benar
    if os.path.exists(file_path):
        print(f"File CSV tersimpan: {file_path}")
    else:
        print("Kesalahan: File CSV tidak berhasil disimpan.")

    # Membaca kembali file CSV dan menampilkan data
    df_loaded = pd.read_csv(file_path)
    print("Data yang diambil dari file CSV:")
    print(df_loaded.head(10))  # Tampilkan 10 data pertama
else:
    print("Tidak ada data yang tersimpan dalam file CSV karena proses scraping gagal.")


Menambang kategori Travel...
Mengambil: https://jatim.tribunnews.com/travel?page=1
Kesalahan saat mengambil https://jatim.tribunnews.com/travel?page=1: 403 Client Error: Forbidden for url: https://jatim.tribunnews.com/travel?page=1
Menambang kategori Sport...
Mengambil: https://jatim.tribunnews.com/sport?page=1
Kesalahan saat mengambil https://jatim.tribunnews.com/sport?page=1: 403 Client Error: Forbidden for url: https://jatim.tribunnews.com/sport?page=1
Jumlah artikel yang diambil: 0
Peringatan: Tidak ada data yang dikumpulkan, pastikan proses scraping berhasil.
Tidak ada data yang tersimpan dalam file CSV karena proses scraping gagal.


In [2]:
df=pd.read_csv("tribunnews_articles.csv")
df.head(1000)

EmptyDataError: No columns to parse from file

In [4]:
import pandas as pd

df = pd.read_csv("tribunnews_articles.csv")

df.head()

EmptyDataError: No columns to parse from file

## **Preprocessing**

Preprocessing adalah proses membersihkan dan mempersiapkan data mentah agar siap digunakan oleh model machine learning. Ini meliputi penanganan data yang hilang, normalisasi, mengubah data kategori menjadi angka, dan membersihkan teks. Tujuannya agar data lebih mudah dipahami dan diolah oleh model untuk hasil yang lebih akurat, Berikut adalah beberapa langkah umum dalam pre-processing teks:

### Cleansing

Proses cleansing data adalah tahap pembersihan teks dari elemen-elemen yang tidak relevan terhadap hasil klasifikasi sentimen. Beberapa komponen yang tidak berpengaruh terhadap sentimen, seperti URL, tag HTML, emoji, simbol, angka, dan tanda baca (~!@#$%^&*{}<>:|), dihapus dari dokumen ulasan. Elemen-elemen tersebut dihilangkan untuk mengurangi kebisingan (noise) dalam data. Setelah dihapus, elemen ini digantikan dengan spasi agar struktur kalimat tetap terjaga. Dengan demikian, data menjadi lebih fokus pada kata-kata yang relevan untuk menentukan sentimen, sehingga membantu meningkatkan akurasi model prediksi sentimen.

In [ ]:
import re
import pandas as pd
import nltk
import string

def remove_url(text):
    #Fungsi untuk menghapus URL dari teks.
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def remove_html(text):
    #Fungsi untuk menghapus tag HTML dari teks.
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)

def remove_emoji(text):

    #Fungsi untuk menghapus emoji dari teks.
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emotikon wajah
        u"\U0001F300-\U0001F5FF"  # simbol & gambar
        u"\U0001F680-\U0001F6FF"  # transportasi & simbol
        u"\U0001F1E0-\U0001F1FF"  # bendera negara
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_numbers(text):
    #Fungsi untuk menghapus angka dari teks.
    return re.sub(r'\d+', '', text)

def remove_symbols(text):
    #Fungsi untuk menghapus simbol dan karakter khusus dari teks.
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

# Asumsikan df adalah DataFrame yang berisi data CNN (judul, berita, tanggal, kategori)
# Contoh: df = pd.read_csv('berita-cnn.csv')

# Terapkan fungsi cleansing untuk kolom 'berita'
df['berita_clean'] = df['isi_berita'].apply(remove_url)
df['berita_clean'] = df['berita_clean'].apply(remove_html)
df['berita_clean'] = df['berita_clean'].apply(remove_emoji)
df['berita_clean'] = df['berita_clean'].apply(remove_symbols)
df['berita_clean'] = df['berita_clean'].apply(remove_numbers)



# Tampilkan beberapa baris dari hasil yang sudah dibersihkan
df.head(5)


### CASE FOLDING

Pada tahap case folding, semua huruf kapital dalam dokumen ulasan diubah menjadi huruf kecil, atau disebut lowercase. Tujuan dari langkah ini adalah untuk menghilangkan redundansi data yang hanya disebabkan oleh perbedaan kapitalisasi. Misalnya, kata "Ekonomi" dan "ekonomi" secara teknis sama dalam analisis teks, namun tanpa case folding, komputer akan menganggapnya berbeda. Dengan mengonversi seluruh teks menjadi huruf kecil, semua variasi penulisan diseragamkan, sehingga mencegah duplikasi penghitungan atau kesalahan dalam interpretasi data.

In [ ]:
def case_folding(text):
    if isinstance(text, str):
      lowercase_text = text.lower()
      return lowercase_text
    else :
      return text

df ['case_folding'] = df['berita_clean'].apply(case_folding)

df.head(5)

### TOKENIZATION

Tokenization adalah tahap di mana setiap kata dalam sebuah dokumen dipecah menjadi unit-unit kata yang lebih kecil, atau disebut token. Proses ini memisahkan kata-kata berdasarkan spasi, sehingga setiap kata yang terpisah oleh spasi dianggap sebagai token tersendiri. Sebagai contoh, kalimat "Upaya agar ekonomi stabil" akan diuraikan menjadi token ["Upaya", "agar", "ekonomi", "stabil"].

In [ ]:
def tokenize(text):
    tokens = text.split()
    return tokens

df['tokenize'] = df['case_folding'].apply(tokenize)

df.head(5)

### STOPWORD REMOVAL

Stopword removal adalah proses menghapus kata-kata yang dianggap tidak penting atau tidak memiliki makna signifikan dalam analisis teks, seperti "dan," "di," "yang," atau "itu." Kata-kata ini sering muncul dalam kalimat tetapi tidak memberikan informasi penting untuk pemrosesan atau analisis lebih lanjut. Dengan menghapus stopwords, data teks menjadi lebih ringkas dan fokus hanya pada kata-kata yang memiliki bobot lebih besar dalam analisis, seperti saat melakukan klasifikasi atau pemodelan teks.

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

In [ ]:
def remove_stopwords(text):
  return [word for word in text if word not in stop_words]

df['stopword_removal'] = df['tokenize'].apply(lambda x: ' '.join(remove_stopwords(x)))


df.to_csv("preprocessing-cnnnews.csv", encoding='utf8', index=False)
df.head(5)

## **TF-IDF (Term Frequency-Inverse Document Frequency)**

TF-IDF adalah metode statistik yang digunakan untuk mengevaluasi pentingnya suatu kata dalam sebuah dokumen relatif terhadap koleksi dokumen lainnya. TF-IDF sering digunakan dalam tugas seperti penggalian teks, penambangan informasi, dan pemodelan pembelajaran mesin berbasis teks.
Term Frequency (TF), yang menghitung seberapa sering sebuah kata muncul dalam dokumen, dan Inverse Document Frequency (IDF), yang menilai seberapa jarang kata tersebut muncul di seluruh dokumen dalam koleksi.

TF-IDF bekerja dengan memberikan bobot lebih tinggi pada kata-kata yang sering muncul dalam sebuah dokumen, tetapi jarang muncul di dokumen lain, sehingga membantu mengidentifikasi kata-kata yang paling relevan.


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv("preprocessing-cnnnews.csv")

# Menginisialisasi TfidfVectorizer
vectorizer = TfidfVectorizer()

# Menghitung TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['stopword_removal'])

In [ ]:
# Mengubah hasilnya menjadi DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_df.head(10)

In [ ]:
# Menginisialisasi TfidfVectorizer dengan normalisasi L2
vectorizer = TfidfVectorizer(norm='l2')

# Menghitung TF-IDF dengan normalisasi L2
tfidf_matrix = vectorizer.fit_transform(df['stopword_removal'])

# Mengubah hasilnya menjadi DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Menampilkan 10 baris pertama
tfidf_df.head(10)


In [ ]:
print(df.columns)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Asumsikan Anda memiliki DataFrame 'df' dengan kolom 'stopword_removal' dan 'kategori'
# Buat fitur (X) dan label (y)
X = tfidf_df
y = df['kategori']  # Pastikan 'kategori' adalah nama kolom yang benar

# Pisahkan dataset menjadi set pelatihan dan pengujian (80% pelatihan, 20% pengujian)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi model Regresi Logistik
model = LogisticRegression(max_iter=1000)

# Latih model
model.fit(X_train, y_train)

# Lakukan prediksi pada set pengujian
y_pred = model.predict(X_test)

# Evaluasi model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nHasil Klasifikasi:")
print(classification_report(y_test, y_pred, zero_division=0))


# Opsional, simpan model untuk digunakan nanti
import joblib
joblib.dump(model, 'model_regresi_logistik.pkl')
